In [1]:
# libs
import datetime
import ctypes
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import logging
import os
from time import sleep
from tkinter import Tk, Label, Button

current_dir = os.getcwd()
input_csv = os.path.join(current_dir,'input.csv')
log_file = os.path.join(current_dir, "consulta_cnpj.log")
with open(input_csv) as f:
    # Lendo todo o arquivo na variável cnpjs
    cnpjs = f.readlines()[1:]
    # Removendo caracteres não numéricos
    cnpjs = [''.join(filter(str.isdigit, cnpj)) for cnpj in cnpjs]
    # Removendo duplicados
    cnpjs = list(set(cnpjs))
print(cnpjs)

['36253414000170', '99999999992103', '26388330000190', '21949888000183', '99999999992022', '21154877000107', '19138890000120', '17217332000125', '99999997984275', '29768219000117', '03389126000198', '20971057000145', '17255670000151', '16636540000104', '18715573000167', '17281106000103', '17155730000164', '22675359000100', '19296342000129', '05465167000141', '16866394000103', '25455858000171', '33224254000142', '28799908000126', '17319831000123', '07256298000144', '19198118000102', '16907746000113', '16695025000197', '17387481000132', '35682930000157', '16625196000140', '09104426000160', '32384286000142', '13252006000106', '11099618000177', '17503475000101', '38486817000194', '18715516000188', '08631821000138', '21229281000129', '08715327000151', '00957404000178', '19701818000160', '19377514000199', '31662649000100', '17161837000115', '16789398000127', '17138140000123', '18715615000160', '10398157000170', '23070071000166', '18715565000110', '05585681000110', '17498205000141', '65172579

In [2]:
def input_ok ():
    # Criando a janela principal do Tkinter
    root = Tk()

    # Adicionando uma Label para exibir a mensagem ao usuário
    label = Label(root, text="Resolva o captcha manualmente e clique em Continuar.")
    label.configure(font={'size': 20})
    label.pack()

    # Função para continuar a execução do script quando o botão for clicado
    def on_button_click():
        # Fechando a janela do Tkinter
        root.destroy()

    # Adicionando um botão para permitir que o usuário continue a execução do script
    button = Button(root, text="Continuar", command=on_button_click)
    button.pack()

    # Exibindo a janela do Tkinter
    root.mainloop()

In [3]:
# Exceção quando a página exibe um alerta
from selenium.common.exceptions import UnexpectedAlertPresentException
import cv2 as cv # Para cortar o print da página
import json # Para exibir o resultado da consulta
from matplotlib import pyplot as plt # Apenas para abrir o captcha automaticamente

try:
    # Pegando a resolução da tela
    user32 = ctypes.windll.user32
    resolucao = user32.GetSystemMetrics(0), user32.GetSystemMetrics(1)

    # Omitindo o navegador na execução
    chrome_options = Options()
    #chrome_options.add_argument('--headless')
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation","enable-logging"])
    chrome_options.add_argument(f'--window-size={resolucao[0]},{resolucao[1]}')
    chrome_options.add_argument("start-maximized")
    chrome_options.add_argument('--enable-cookies')
    chrome_options.add_argument("--log-level=3")
    # Desabilitando as mensagens do selenium no terminal (apenas erros serão exibidos)
    chrome_options.add_argument("--log-level=3")
    # Desabilitando a abertura automática de uma janela do navegador com a página da receita
    #options.add_argument("--headless")

    # Parâmetros do navegador
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

    # Fazendo o GET da página da receita
    URL = "http://servicos.receita.fazenda.gov.br/Servicos/cnpjreva/Cnpjreva_Solicitacao_CS.asp"
    driver.get(URL)

    # Encontrando o campo de CNPJ
    input_cnpj = driver.find_element("id", "cnpj")
    # Encontrando o campo de digitar o captcha
    input_captcha = driver.find_element("id", "txtTexto_captcha_serpro_gov_br")

    # Processo de capturar o captcha da página:

    # Salvando o print da página
    driver.save_screenshot("screenshot.png")
    # Encontrando a imagem na página
    img = driver.find_element("id", "imgCaptcha")
    # Armazenando as coordenadas (x,y) do captcha na tela
    loc = img.location
    # Realizando o processo de cortar a imagem do print
    image = cv.imread('screenshot.png')
    desloc_x = 185
    desloc_y = 55
    cropped_image = image[loc['y']:loc['y']+desloc_y, loc['x']:loc['x']+desloc_x]
    cv.imwrite('captcha.png', img)
    # Exibindo o captcha automaticamente
    img_captcha = plt.imread('captcha.png')
    plt.imshow(img_captcha)
    plt.axis('off')
    plt.show(block=False)

    # Lendo do terminal o CNPJ e o captcha digitados
    cnpj = input("Digite o CNPJ (somente números): ")
    captcha = input("Digite os caracteres da imagem exibida: ")

    # Escrevendo na página o CNPJ e captcha recebidos via terminal
    driver.execute_script("arguments[0].value='{}';".format(cnpj), input_cnpj)
    driver.execute_script("arguments[0].value='{}';".format(captcha), input_captcha)

    # Realizando o submit do formulário da página da receita
    retorno = driver.find_element("id", "frmConsulta").submit()

    # Processo de retirar as informações do resultado da consulta:

    # Verifica se o resultado da consulta foi uma mensagem de erro
    temp = driver.find_elements("tag name", "b")
    verifica = []
    for t in temp:
        verifica.append(t.text)
    if len(verifica) == 1:
        print("\n{}: Captcha inválido".format(verifica[0]))
    else: # Se não foi uma mensagem de erro, realiza o processo de obtenção dos resultados
        linha = driver.find_elements("tag name", "font")
        for i in range(len(linha)):
            if linha[i].text == "NÚMERO DE INSCRIÇÃO":
                insc = linha[i+1].find_elements("tag name", "b")
                final_cnpj = insc[0].text
                tipo_empresa = insc[1].text
            elif linha[i].text == "DATA DE ABERTURA":
                data_abertura = linha[i+1].find_element("tag name", "b").text
            elif linha[i].text == "NOME EMPRESARIAL":
                nome_empresarial = linha[i+1].find_element("tag name", "b").text
            elif linha[i].text == "TÍTULO DO ESTABELECIMENTO (NOME DE FANTASIA)":
                nome_fantasia = linha[i+1].find_element("tag name", "b").text
            elif linha[i].text == "LOGRADOURO":
                logradouro = linha[i+1].find_element("tag name", "b").text
            elif linha[i].text == "NÚMERO":
                numero = linha[i+1].find_element("tag name", "b").text
            elif linha[i].text == "COMPLEMENTO":
                complemento = linha[i+1].find_element("tag name", "b").text
            elif linha[i].text == "CEP":
                cep = linha[i+1].find_element("tag name", "b").text
            elif linha[i].text == "BAIRRO/DISTRITO":
                bairro = linha[i+1].find_element("tag name", "b").text
            elif linha[i].text == "MUNICÍPIO":
                municipio = linha[i+1].find_element("tag name", "b").text
            elif linha[i].text == "UF":
                estado = linha[i+1].find_element("tag name", "b").text
            elif linha[i].text == "ENDEREÇO ELETRÔNICO":
                email = linha[i+1].find_element("tag name", "b").text
            elif linha[i].text == "TELEFONE":
                telefone = linha[i+1].find_element("tag name", "b").text
            elif linha[i].text == "SITUAÇÃO CADASTRAL":
                situacao_cadastral = linha[i+1].find_element("tag name", "b").text
            elif linha[i].text == "DATA DA SITUAÇÃO CADASTRAL":
                data_situacao_cadastral = linha[i+1].find_element("tag name", "b").text

        # Monta o JSON que será retornado
        final_json = {
            "cnpj": final_cnpj,
            "tipo_empresa": tipo_empresa,
            "data_abertura": data_abertura,
            "nome_empresarial": nome_empresarial,
            "nome_fantasia": nome_fantasia,
            "endereco": {
                "logradouro": logradouro,
                "numero": numero,
                "complemento": complemento,
                "cep": cep,
                "bairro": bairro,
                "municipio": municipio,
                "estado": estado
            },
            "email": email,
            "telefone": telefone,
            "situacao_cadastral": situacao_cadastral,
            "data_situacao_cadastral": data_situacao_cadastral
        }

        # Exibe o resultado formatado
        print("\nRESULTADO DA CONSULTA:")
        print(json.dumps(final_json, indent=4, sort_keys=False))

    # Encerra a sessão do selenium
    driver.quit()
except UnexpectedAlertPresentException as e:
    print(e)
except Exception as e:
    print(e)

OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'imwrite'
> Overload resolution failed:
>  - img is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'img'



In [ ]:

# gerando log
logging.basicConfig(level=logging.INFO, filename=log_file,
                    encoding='utf-8', format="%(asctime)s - %(levelname)s - %(message)s")

# Pegando a resolução da tela

user32 = ctypes.windll.user32
resolucao = user32.GetSystemMetrics(0), user32.GetSystemMetrics(1)

# Omitindo o navegador na execução
chrome_options = Options()
#chrome_options.add_argument('--headless')
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation","enable-logging"])
chrome_options.add_argument(f'--window-size={resolucao[0]},{resolucao[1]}')
chrome_options.add_argument("start-maximized")
chrome_options.add_argument('--enable-cookies')
chrome_options.add_argument("--log-level=3")

# Parâmetros do navegador
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
for cnpj in cnpjs:
    driver.get(f"https://solucoes.receita.fazenda.gov.br/Servicos/cnpjreva/Cnpjreva_Solicitacao.asp")
    driver.implicitly_wait(15)
    driver.maximize_window()
    actions = ActionChains(driver, duration=100)
    
    # Inputa o cnpj
    driver.find_element(By.NAME,"cnpj").send_keys(cnpj)

    # Tenta resolver o captcha
    # Mudando para o iframe
    iframe = driver.find_element(By.XPATH,'/html/body/div/div[1]/div[1]/div/div/form/div[1]/div[2]/div/iframe')
    driver.switch_to.frame(iframe)

    # Clicando no captcha
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, "checkbox"))).click()

    # Pausando a execução do script até que o usuário pressione Enter
    #input("Pressione `Enter` depois de resolver o captcha manualmente.")
    input_ok()

    # Mudando de volta para a janela principal
    driver.switch_to.default_content()

    # Clicando no  botão consultar
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,'/html/body/div/div[1]/div[1]/div/div/form/div[3]/div/button[1]'))).click()
    
    break
sleep(5)
#driver.close()